In [1]:
#############
## Imports ##
#############

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')


STYLE = "#ffffff"

import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import time

import tensorflow as tf
from tensorflow.keras import Sequential, layers
import keras

from scipy.fft import dst, dct, fft

from IPython import display

ModuleNotFoundError: No module named 'imageio'

In [ ]:
######################
## Generic GAN data ##
######################

BUFFER_SIZE = 60000
BATCH_SIZE = 256

(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data() # No need of test data

train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')

train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]

train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE) # Batch and shuffle the data

In [ ]:
#########################
## Spectogram GAN data ##
#########################

image = np.random.randint(10, size = (128,1024))

fig, axes = plt.subplots(1, 1, figsize=(20, 20), facecolor = (STYLE))

axes.imshow(image, cmap="gray")

In [ ]:
 # def f(w1, w2):
 #     return 3 * w1 ** 2 + 2 * w1 * w2

In [ ]:
# w1, w2 = tf.Variable(5.), tf.Variable(3.)
# with tf.GradientTape() as tape:
#     z = f(w1, w2)

# gradients = tape.gradient(z, [w1, w2])

In [ ]:
# gradients

In [ ]:
# Architecture guidelines for stable Deep Convolutional GANs (no mathematical reason, just trial and error, Radford et al., 2015)

# Replace any pooling layers with strided convolutions (discriminator) and fractional-strided convolutions (generator).

# Use batchnorm in both the generator and the discriminator.

# Remove fully connected hidden layers for deeper architectures.

# Use ReLU activation in generator for all layers except for the output, which uses Tanh.

# Use LeakyReLU activation in the discriminator for all layers.

# create, train, validate, predict.

In [ ]:
######################
## Model: Generator ##
######################

# The generator uses tf.keras.layers.Conv2DTranspose (upsampling) layers to produce an image from a seed (random noise). 

def make_generator_model():
    model = tf.keras.Sequential() # Initialize Sequential model
    # The Sequential model is a straight line. You keep adding layers, every new layer takes the output of the previous layer. You cannot make creative graphs with branches
    # The functoinal API Model is completely free to have as many ramifications, inputs and outputs as you need
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,))) # shape 100 noise vector, 7*7*256 flat layer to reshape [7,7,256] | 7 width 7 height 256 channels
    model.add(layers.BatchNormalization()) # BatchNormalization doesn't require bias, makes the model faster and more stable
    model.add(layers.ReLU()) # LeakyReLU
    model.add(layers.Reshape((7, 7, 256))) # reshape [7,7,256] 
    assert model.output_shape == (None, 7, 7, 256) # None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False)) # 128 Filters... to be the number of channels of the output, (5,5) kernel
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model

In [ ]:
generator = make_generator_model()

noise = tf.random.normal([7, 1, 100]) # Random input vecto [number of samples, Width, Height]

fig, axes = plt.subplots(1, 7, figsize=(30, 30), facecolor = (STYLE))

axes[0].imshow(generator(noise[0, :, :], training=False)[0, :, :, 0], cmap='gray') # INPUT | NOISE [[number of samples, Width, Height] OUTPUT | PLOT [number of samples, Width, Height, Channels]
axes[1].imshow(generator(noise[1, :, :], training=False)[0, :, :, 0], cmap='gray') # INPUT | NOISE [[number of samples, Width, Height] OUTPUT | PLOT [number of samples, Width, Height, Channels]
axes[2].imshow(generator(noise[2, :, :], training=False)[0, :, :, 0], cmap='gray') # INPUT | NOISE [[number of samples, Width, Height] OUTPUT | PLOT [number of samples, Width, Height, Channels]
axes[3].imshow(generator(noise[3, :, :], training=False)[0, :, :, 0], cmap='gray') # INPUT | NOISE [[number of samples, Width, Height] OUTPUT | PLOT [number of samples, Width, Height, Channels]
axes[4].imshow(generator(noise[4, :, :], training=False)[0, :, :, 0], cmap='gray') # INPUT | NOISE [[number of samples, Width, Height] OUTPUT | PLOT [number of samples, Width, Height, Channels]
axes[5].imshow(generator(noise[5, :, :], training=False)[0, :, :, 0], cmap='gray') # INPUT | NOISE [[number of samples, Width, Height] OUTPUT | PLOT [number of samples, Width, Height, Channels]
axes[6].imshow(generator(noise[6, :, :], training=False)[0, :, :, 0], cmap='gray') # INPUT | NOISE [[number of samples, Width, Height] OUTPUT | PLOT [number of samples, Width, Height, Channels]

In [ ]:
# Use the (as yet untrained) generator to create an image

generator = make_generator_model() # function defined in previous cell

generated_image = generator(noise[0, :, :], training=False)

# pd.DataFrame(generated_image[0, :, :, 0])

In [ ]:
##########################
## Model: Discriminator ##
##########################

# The discriminator is a CNN-based image classifier it uses tf.keras.layers.Conv2D to classify images as real or fake

def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

In [ ]:
discriminator = make_discriminator_model()

decision = discriminator(generated_image)

print(decision) # 50 / 50 not trained, will be trained to generate positive values for real pictures and negative for generated ones

In [ ]:
##################################
## Model Compile: Loss Function ##
##################################

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True) # we take a BinaryCrossentropy

# Binary cross entropy compares each of the predicted probabilities to actual class output which can be either 0 or 1
# Binary Cross Entropy is the negative average of the log of corrected predicted probabilities

In [ ]:
########################
## Discriminator loss ##
########################

# quantifies how well the discriminator is able to distinguish real images from generated 

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    # compares the predictions of the discriminator over real images to a matrix of [1s] | must have a tendency/likelihood to 1
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    # compares the predictions of the discriminator over generated images to a matrix of [0s] | must have a tendency/likelihood to 0
    total_loss = real_loss + fake_loss
    return total_loss # Total loss

In [ ]:
####################
## Generator loss ##
####################

# quantifies how well it was able to trick the discriminator, if the generator is performing well, the discriminator will classify the fake images as real (1). 

def generator_loss(fake_output):
    binary_cross_entropy = cross_entropy(tf.ones_like(fake_output), fake_output)
    # the generator's output need to have a tendency to 1, We compare the discriminators decisions on the generated images to an array of [1s]
    return binary_cross_entropy

In [ ]:
##############################
## Model Compile: Optimizer ##
##############################

# Two different optimizers since we train two separate networks:

generator_optimizer = tf.keras.optimizers.Adam(1e-4) # SGD INSTEAD???   (Radford et al., 2015)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4) # SGD INSTEAD???  (Radford et al., 2015)

In [ ]:
######################
## Save checkpoints ##
######################

# save and restore models, which can be helpful in case a long running training task is interrupted.

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
##############################
## Define the training loop ##
##############################

EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16
seed = tf.random.normal([num_examples_to_generate, noise_dim])
# You will reuse this seed overtime (so it's easier) to visualize progress in the animated GIF)

In [ ]:
noise = tf.random.normal([BATCH_SIZE, noise_dim])

noise.shape

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function

####################
## Training steps ##
####################

def train_step(images): # train for just ONE STEP aka one forward and back propagation
    noise = tf.random.normal([BATCH_SIZE, noise_dim]) # generate the noises [batch size, latent space 100 dimention vector]

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape: # get the gradient for each parameter for this step
      generated_images = generator(noise, training=True) # iterates over the noises

      real_output = discriminator(images, training=True) # trains discriminator based on labeled real pics
      fake_output = discriminator(generated_images, training=True) # trains discriminator based on labeled generated pics
    # why it doesnt traing all at ones

      gen_loss = generator_loss(fake_output) # calculating the generator loss function previously defined
      disc_loss = discriminator_loss(real_output, fake_output) # calculating the descrim loss function previously defined

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    # saving the gradients of each trainable variable of the generator
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    # saving the gradients of each trainable variable of the discriminator

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    # applying the gradients on the trainable variables of the generator to update the parameters
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    # applying the gradients on the trainable variables of the generator to update the parameters

In [ ]:
###################
## Training loop ##
###################

# training loop itself using train_step function previously defined 

def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch) # divides the data set in batches rains the step on the batch

    # Produce images 
    display.clear_output(wait=True) # clearing output !!!TO BE CHECKED!!!
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix) # saving weights and biases previously calculated by the train step gradients

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)

# train(train_dataset, 10)

In [ ]:
###############################
## Generate and save images  ##
###############################

def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4, 4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [ ]:
train(train_dataset, EPOCHS)

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
############################
## Display a single image ##
############################

def display_image(epoch_no): # using the epoch number
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
display_image(EPOCHS)

In [ ]:
##################
## Animated GIF ##
##################

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)